In [1]:
import numpy as np
import pandas as pd
import os
import sys
import random
import keras
import tensorflow as tf
import json
import bert
from bert import tokenization

Using TensorFlow backend.
c:\users\subburam.subbiah\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\subburam.subbiah\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\subburam.subbiah\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint1

In [2]:
from keras_bert.bert import get_model
from keras_bert.loader import load_trained_model_from_checkpoint
from keras.optimizers import Adam
adam = Adam(lr=2e-5,decay=0.01)
maxlen = 75
BERT_PRETRAINED_DIR = 'D:/uncased_L-12_H-768_A-12'
config_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
checkpoint_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
model = load_trained_model_from_checkpoint(config_file, checkpoint_file, training=True,seq_len=maxlen)

In [8]:
from keras.layers import Dense,Input,Flatten,concatenate,Dropout,Lambda
from keras.models import Model
import keras.backend as K
import re
import codecs

sequence_output  = model.layers[-6].output
pool_output = Dense(5, activation='softmax',kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),name = 'real_output')(sequence_output)
model3 = Model(inputs=model.input, outputs=pool_output)
model3.compile(loss='sparse_categorical_crossentropy', optimizer=adam)

In [9]:
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for i in range(example.shape[0]):
      tokens_a = tokenizer.tokenize(example[i])
      if len(tokens_a)>max_seq_length:
        tokens_a = tokens_a[:max_seq_length]
        longer += 1
      one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
      all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)

In [10]:
nb_epochs=25
bsz = 32
dict_path = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
def create_tokenizer(vocab_file, do_lower_case=False):
    return tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)
tokenizer = create_tokenizer(dict_path, do_lower_case=False)
print('build tokenizer done')

build tokenizer done


In [11]:
model3.load_weights('D:/15-Feb/bert_weights.h5')

In [18]:
test_df = pd.read_excel('D:/15-Feb/val.xlsx')
test_df['content'] = test_df['content'].str.lower()
test_df = test_df['content'].dropna()
test_df=test_df.iloc[1:10]
eval_lines = test_df.values

In [25]:
token_input2 = convert_lines(eval_lines,maxlen,tokenizer)
seg_input2 = np.zeros((token_input2.shape[0],maxlen))
mask_input2 = np.ones((token_input2.shape[0],maxlen))
emotions = model3.predict([token_input2, seg_input2, mask_input2],verbose=1,batch_size=bsz)

0
9/9 [==============================] - 1s 152ms/step


In [26]:
emotions=np.argmax(emotions,axis=1)
df_emotions=pd.DataFrame(emotions)
test_df=pd.DataFrame(test_df)

In [33]:
test_df['emotions']=df_emotions

In [36]:
test_df.to_csv('test.csv')